# Projet 5 DataV2 BottleNeck
## Notebook 1 : Analyses préparatoires<br>

[lisezmoi.txt](lisezmoi.txt) informations sur les versions des bibliothèques utilisées<br>
[lien vers le notebook 2 : Nettoyage](nettoyage.ipynb)<br>
[lien vers le notebook 3 : Analyse](analyse.ipynb)<br>
Ce fichier regroupe diverses analyses pour avoir un aperçu des données de bases.<br>
Certaines analyses ne sont là qu'à titre informatif et n'interfèrent pas avec les analyses demandées<br>
## Analyses des fichiers de base

### points importants:
#####  fichier erp:
_[clé possible](#erp_cle) pour jointure product_id<br>
_on_sale_web est un [booléen](#erp_bool) codé en entier (0 = False et 1 = True)<br>
_une [incohérence](#erp_incoherence) entre la colonne stock_quantity et la colonne stock_status<br>
#####  fichier web:
_[PAS DE CLE POUR JOINTURE](#web_cle) à créer pendant le nettoyage<br>
_des [colonnes remplies de 0](#web_colonne0) et d'autres inutiles pour les analyses demandées<br>
_des [NaN](#web_NaN) dans les colonnes à vérifier pendant le nettoyage<br>
##### fichier liaison:
_[valeur manquante](#id_web_valeur_manquante) dans id_web (normal tous les produits ne sont pas forcement en vente sur le site)<br>
_[pas de valeurs manquantes](#id_web_product_id) (NaN) dans product_id<br>

## code python
### importation des bibliothèques

In [1]:
import warnings # pour gerer les alarmes de openyxl (inclus dans pandas)
import pandas as pd

#fonction perso se trouvant dans le dossier fonction (description à la fin du notebook)
import fonction.fonction_perso as perso 

### Importation des fichiers

In [2]:
# openpyxl me met un warning car le systeme ne reconnait pas l'extension excel (linux power)
warnings.simplefilter("ignore")

erp = pd.read_excel('donnee/erp.xlsx')
vente_web = pd.read_excel('donnee/web.xlsx')
liaison = pd.read_excel('donnee/liaison.xlsx')

### Analyse du fichier ERP

In [3]:
perso.analyseDataFrame(erp)

################################# INFO  #################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      825 non-null    int64  
 1   onsale_web      825 non-null    int64  
 2   price           825 non-null    float64
 3   stock_quantity  825 non-null    int64  
 4   stock_status    825 non-null    object 
dtypes: float64(1), int64(3), object(1)
memory usage: 32.4+ KB
################################# DESCRIBE  #################################


,product_id,onsale_web,price,stock_quantity,stock_status
count,825.000000,825.000000,825.000000,825.000000,825
unique,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,instock
freq,NaN,NaN,NaN,NaN,638
mean,5162.597576,0.869091,32.415636,26.496970,NaN
std,902.644635,0.337506,26.795849,45.892422,NaN
min,3847.000000,0.000000,5.200000,0.000000,NaN
25%,4348.000000,1.000000,14.600000,1.000000,NaN
50%,4907.000000,1.000000,24.400000,11.000000,NaN
75%,5805.000000,1.000000,42.000000,34.000000,NaN


################################# HEAD  #################################


,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


################################# DOUBLONS  #################################


,product_id,onsale_web,price,stock_quantity,stock_status


################################## recherche de clé pour merge ###################
la colonne product_id est une clé


<a id='erp_cle'></a>
#### recherche de clé possible pour jointure

In [4]:
perso.detection_clé(erp)

la colonne product_id est une clé


<a id='erp_bool'></a>
#### Vérification cohérence des données
##### verification onsale_web

In [5]:
#verification coherence de onsale_web
print("si affichage vide il n'y a pas de valeur superieure à 1 ni",
      "de valeur inferieure à 0\nil s'agit d'un booléen")
display(erp[(erp['onsale_web']>1) & (erp['onsale_web']<0)])

si affichage vide il n'y a pas de valeur superieure à 1 ni de valeur inferieure à 0
il s'agit d'un booléen


,product_id,onsale_web,price,stock_quantity,stock_status


##### Vérification de la colonne stock_quantity

In [6]:
print("stock superieur à 0")
display(erp[erp['stock_quantity']>0]) #verification si on a du stock quelque part.
print("stock inferieur à 0")
display(erp[erp['stock_quantity']<0]) #verification si on a pas de valeur negative 

stock superieur à 0


,product_id,onsale_web,price,stock_quantity,stock_status
6,4041,1,32.7,15,instock
7,4042,1,31.2,34,instock
8,4043,1,60.0,12,instock
9,4045,1,42.6,66,instock
11,4047,1,18.3,36,instock
...,...,...,...,...,...
820,7203,0,45.0,30,instock
821,7204,0,45.0,9,instock
822,7247,1,54.8,23,instock
823,7329,0,26.5,14,instock


stock inferieur à 0


,product_id,onsale_web,price,stock_quantity,stock_status


<a id='erp_incoherence'></a>
### cohérence entre la colonne stock_quantity et la colonne stock_status

In [7]:
#creation d'une colonne pour verifier la coherence entre le stock et son statut
erp['coherence stock'] = (((erp['stock_status'] == 'outofstock')
                          & (erp['stock_quantity']<=0)) 
                          |((erp['stock_status'] == 'instock')
                          & (erp['stock_quantity']>0))
                         )   
print("affichage des données incoherentes")
display(erp[erp['coherence stock'] == False])

affichage des données incoherentes


,product_id,onsale_web,price,stock_quantity,stock_status,coherence stock
443,4954,1,25.0,0,instock,False


##### Il faudra garder à l'esprit cette erreur pour une éventuelle correction ou une future analyse 
#### Pour les analyses demandées, le stock ainsi que le status ne sont pas nécessaires

<a id='web_colonne0'></a>
<a id='web_NaN'></a>
### Analyse du fichier vente web

In [8]:
perso.analyseDataFrame(vente_web)

################################# INFO  #################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   sku                    1428 non-null   object        
 1   virtual                1513 non-null   int64         
 2   downloadable           1513 non-null   int64         
 3   rating_count           1513 non-null   int64         
 4   average_rating         1430 non-null   float64       
 5   total_sales            1430 non-null   float64       
 6   tax_status             716 non-null    object        
 7   tax_class              0 non-null      float64       
 8   post_author            1430 non-null   float64       
 9   post_date              1430 non-null   datetime64[ns]
 10  post_date_gmt          1430 non-null   datetime64[ns]
 11  post_content           0 non-null      float64 

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
count,1428,1513.0,1513.0,1513.0,1430.0,1430.000000,716,0.0,1430.000000,1430,...,1430,1430,1430,0.0,1430.0,1430,1430.0,1430,714,1430.0
unique,714,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,716,NaN,NaN,NaN,NaN,1430,NaN,2,1,NaN
top,bon-cadeau-25-euros,NaN,NaN,NaN,NaN,NaN,taxable,NaN,NaN,NaN,...,bon-cadeau-de-25-euros,NaN,NaN,NaN,NaN,https://www.bottle-neck.fr/?post_type=product&...,NaN,product,image/jpeg,NaN
freq,2,NaN,NaN,NaN,NaN,NaN,716,NaN,NaN,NaN,...,2,NaN,NaN,NaN,NaN,1,NaN,716,714,NaN
mean,NaN,0.0,0.0,0.0,0.0,4.006993,NaN,NaN,1.998601,2018-08-22 03:22:17.090908928,...,NaN,2020-06-20 13:59:29.781818368,2020-06-20 12:06:02.509090816,NaN,0.0,NaN,0.0,NaN,NaN,0.0
min,NaN,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,1.000000,2018-02-08 12:58:52,...,NaN,2018-02-20 15:19:23,2018-02-20 14:19:23,NaN,0.0,NaN,0.0,NaN,NaN,0.0
25%,NaN,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,2.000000,2018-02-27 20:01:12.500000,...,NaN,2020-06-18 10:45:05.249999872,2020-06-18 08:45:05.249999872,NaN,0.0,NaN,0.0,NaN,NaN,0.0
50%,NaN,0.0,0.0,0.0,0.0,1.000000,NaN,NaN,2.000000,2018-04-19 14:56:05,...,NaN,2020-08-04 09:30:06,2020-08-04 07:30:06,NaN,0.0,NaN,0.0,NaN,NaN,0.0
75%,NaN,0.0,0.0,0.0,0.0,4.000000,NaN,NaN,2.000000,2019-01-31 14:35:47,...,NaN,2020-08-25 10:32:32,2020-08-25 08:32:32,NaN,0.0,NaN,0.0,NaN,NaN,0.0
max,NaN,0.0,0.0,0.0,0.0,96.000000,NaN,NaN,2.000000,2020-07-20 11:00:00,...,NaN,2020-08-27 18:55:03,2020-08-27 16:55:03,NaN,0.0,NaN,0.0,NaN,NaN,0.0


################################# HEAD  #################################


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,bon-cadeau-25-euros,0,0,0,0.0,10.0,taxable,NaN,1.0,2018-06-01 13:53:46,...,bon-cadeau-de-25-euros,2018-06-01 14:13:57,2018-06-01 12:13:57,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
1,15298,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-08 12:58:52,...,pierre-jean-villa-saint-joseph-preface-2018,2019-12-30 09:30:29,2019-12-30 08:30:29,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
2,15296,0,0,0,0.0,0.0,taxable,NaN,2.0,2018-02-08 13:49:41,...,pierre-jean-villa-saint-joseph-tilde-2017,2019-12-21 09:00:17,2019-12-21 08:00:17,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,15300,0,0,0,0.0,0.0,taxable,NaN,2.0,2018-02-08 14:08:36,...,pierre-jean-villa-croze-hermitage-accroche-coe...,2020-06-26 18:15:03,2020-06-26 16:15:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,19814,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-09 14:01:05,...,pierre-jean-villa-igp-gamine-2018,2020-01-04 16:36:01,2020-01-04 15:36:01,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0


################################# DOUBLONS  #################################


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
179,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
794,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
795,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


################################## recherche de clé pour merge ###################
pas de clé


<a id='web_cle'></a>
#### détection si clé pour jointure

In [9]:
 #si affiche rien le fichier ne peut avoir de clé valide (pour l'instant)
perso.detection_clé(vente_web) 

pas de clé


#### tax status
indique les éléments uniques de la colonne

In [10]:
print(vente_web['tax_status'].unique())

['taxable' nan]


#### taxclass
indique les éléments uniques de la colonne

In [11]:
print(vente_web['tax_class'].unique())

[nan]


#### total sale

In [12]:
print("valeur unique\n")
print(vente_web['total_sales'].unique())
print("\nla valeur moyenne du nombre de vente est de ",round(vente_web['total_sales'].mean(),2))
print("\ntableau du nombre d'apparition des valeurs")
display(vente_web['total_sales'].value_counts(ascending = False,dropna=False))

valeur unique

[10.  6.  0.  3.  7. 14.  1. 23. 18. 11.  9. 20. 19. 62.  4. 12.  5.  2.
 40. 30. 87. 36. 43. 13. 17. 42.  8. 46. 16. 15. nan 96. 32. 38. 29. 33.
 22. 24. 26. 21. 41. 37.]

la valeur moyenne du nombre de vente est de  4.01

tableau du nombre d'apparition des valeurs


0.0     660
1.0     174
3.0     104
NaN      83
2.0      80
4.0      60
6.0      50
5.0      44
7.0      38
12.0     30
10.0     20
8.0      20
9.0      20
18.0     14
11.0     14
13.0     12
19.0     10
15.0      8
14.0      8
38.0      4
17.0      4
22.0      4
16.0      4
24.0      4
36.0      4
30.0      4
23.0      4
20.0      4
42.0      2
29.0      2
41.0      2
21.0      2
26.0      2
33.0      2
62.0      2
43.0      2
32.0      2
96.0      2
40.0      2
87.0      2
46.0      2
37.0      2
Name: total_sales, dtype: int64

#### Analyse de post_name :  les noms des bouteilles 

In [13]:
print("Nombre de nom unique dans post_name:",len(vente_web['post_name'].unique()))
print("Nombre de nom unique dans sku:",len(vente_web['sku'].unique()))

Nombre de nom unique dans post_name: 717
Nombre de nom unique dans sku: 715


##### Analyse des erreurs entre les noms des bouteilles et du sku

In [14]:
#création d'un dataframe sku, postname
sku_name = vente_web[['sku','post_name']]
#enlevé les doublons dans le data sku postname
sku_name = sku_name.drop_duplicates()

print(sku_name.info())

print("\nnombre de valeurs uniques de la colonne sku :",len(vente_web['sku'].unique()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717 entries, 0 to 798
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sku        714 non-null    object
 1   post_name  716 non-null    object
dtypes: object(2)
memory usage: 16.8+ KB
None

nombre de valeurs uniques de la colonne sku : 715


##### Pas de clé déterminée dans le fichier web avant nettoyage. A faire avant la liaison

#### Doulons présents à analyser
affichage des doublons sans les NaN dans la colonne sku

In [15]:
doublons_vente_web = vente_web[vente_web.duplicated()]
display(doublons_vente_web[~doublons_vente_web['sku'].isna()])
#nettoyage des données
del doublons_vente_web

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count


### Analyse du fichier liaison ERP web

In [16]:
perso.analyseDataFrame(liaison)

################################# INFO  #################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  825 non-null    int64 
 1   id_web      734 non-null    object
dtypes: int64(1), object(1)
memory usage: 13.0+ KB
################################# DESCRIBE  #################################


,product_id,id_web
count,825.000000,734.0
unique,NaN,734.0
top,NaN,15298.0
freq,NaN,1.0
mean,5162.597576,NaN
std,902.644635,NaN
min,3847.000000,NaN
25%,4348.000000,NaN
50%,4907.000000,NaN
75%,5805.000000,NaN


################################# HEAD  #################################


,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815


################################# DOUBLONS  #################################


,product_id,id_web


################################## recherche de clé pour merge ###################
la colonne product_id est une clé


#### Vérifier si il n'y pas de doublons dans la colonne product_id et id_web

In [17]:
print("doublons dans la colonne 'product_id'")
display(liaison[liaison['product_id'].duplicated()])
print("doublons dans la colonne 'id_web'")
display(liaison[liaison['id_web'].duplicated()])

doublons dans la colonne 'product_id'


,product_id,id_web


doublons dans la colonne 'id_web'


,product_id,id_web
49,4090,NaN
50,4092,NaN
119,4195,NaN
131,4209,NaN
151,4233,NaN
...,...,...
817,7196,NaN
818,7200,NaN
819,7201,NaN
820,7203,NaN


<a id ='id_web_product_id'></a>

In [18]:
print("NaN dans la colonne 'product_id'")
display(liaison[liaison['product_id'].isna()])

NaN dans la colonne 'product_id'


,product_id,id_web


<a id='id_web_valeur_manquante'></a>

In [19]:
print("Nan dans la colonne 'id_web'")
display(liaison[liaison['id_web'].isna()])

Nan dans la colonne 'id_web'


,product_id,id_web
19,4055,NaN
49,4090,NaN
50,4092,NaN
119,4195,NaN
131,4209,NaN
...,...,...
817,7196,NaN
818,7200,NaN
819,7201,NaN
820,7203,NaN


In [20]:
#fonction incluse dans le fichier perso
help(perso)

Help on module fonction.fonction_perso in fonction:

NAME
    fonction.fonction_perso

FUNCTIONS
    analyseDataFrame(df)
        affiche un resumé des Dataframes info describe head et verification doublons
        
        Args:
            df : dataframe à analyser as dataFrame
        
        Returns:
            None
    
    categoriser(data, nom_col_recherche, mot_cle, nom_categorie, nom_col_categorie='categorie', regex=False)
        recherche un mot dans une string et enregistre un nom de categorie dans la colonne categorie (par defaut)
        Args:
            data : dataframe a categoriser as DataFrame
            nom_col_recherche : colonne où rechercher as string
            nom_categorie : nom de la catégorie as string
            nom_col_categorie : nom de la colonne cat"egorie a créer as string
        Returns :
            None
    
    courbe_Lorentz(plt_ax, df, colonne, titre='', titreX='', titreY='')
        configure une courbe de Lorentz sur l'axe matplotlib
    